In [1]:
import numpy as np
import pandas as pd
import torch
import pickle
import re
import requests
from bs4 import BeautifulSoup
import collections
from itertools import chain
from collections import Counter
import torch.nn as nn
import glob
import random
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset, random_split
from tqdm import tqdm
import time

import spacy
from spacy import displacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.util import filter_spans
nlp = spacy.load("en_core_web_lg")

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
import sys
sys.path.insert(0, '../../src/models/')
sys.path.insert(0, '../../src/features/')
import predict_model
model = predict_model.loadBERT("../../models/", 'saved_weights_inf_FIXED_boot.pt')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

%matplotlib inline

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CPU Success


In [2]:
def SpanPredictor(span, pred_values=False):
    
    """
    Uses a trained bert classifier to see if a span
    belongs to a species description or otherwise.
    """
         
    with torch.no_grad():
        # Tokenize input
        inputs = tokenizer(span, return_tensors="pt", truncation=True)
        # Predict class
        outputs = model(inputs['input_ids'], inputs['attention_mask'])
        # Get prediction values
        exps = torch.exp(outputs)
        # Get class
        span_class = exps.argmax(1).item()

        # Print the prediction values
        if pred_values:
            return span_class, exps[0]
        else:
            return span_class

In [ ]:
#google_results = pickle.load(open('google_test.pkl', 'rb'))
#ddg_results = pickle.load(open('ddg_test.pkl', 'rb'))
combi_results = pickle.load(open('combi_test.pkl', 'rb'))

In [ ]:
# Init dict
data = collections.defaultdict(list)


regexes = [
    r' \(\d+.+?Close\n\t\n\)',
    r'\(.+\)',
    r'\[.+\]',
    r'\xa0'
]

for species, URL_list in tqdm(combi_results.items()):
    URL_list = list(set(URL_list))
    for URL in URL_list:
        try:
            #print(URL)
            page = requests.get(URL, timeout=5)
            # Skip PDF files for now
            if page.headers['Content-Type'].startswith('application/pdf'):
                continue
            # Soup the result
            soup = BeautifulSoup(page.content, 'html.parser')
            # Skip Embedded PDF's
            if 'pdf' in soup.title.text.lower():
                continue

            ## CLEANING
            # Get the cleaned strings
            #text_list = [text for text in soup.stripped_strings]
            
            text_list = [text.strip() for text in soup.get_text().split('\n')
                        if len(text) > 2]
            # Init
            text_list_cleaned = []
            # Loop over current list and clean more
            for idx, text in enumerate(text_list):
                # Replace rubbish
                text = text.replace('\n', "")
                text = text.replace('\r', "")
                text_list_cleaned.append(text)

            # Create string
            cleaned_text = '. '.join(text_list_cleaned)
            cleaned_text = re.sub(r'\.\.', '. ', cleaned_text, flags=re.DOTALL)
            ## SPACY
            # nlp the text
            doc = nlp(cleaned_text)
            # Extract the sents
            sentences = [i for i in doc.sents]

            # Loop over the individual sentences
            for sentence in sentences:                    
                # Create string object
                sentence_str = str(sentence)
                # Loop over the regexes
                for regex in regexes:
                    # Clean
                    sentence_str = re.sub(regex, '', sentence_str, flags=re.DOTALL)
                # Skip single crap
                if len(sentence_str[0:-1].split()) <= 5:
                    continue
                # Skip measurement crap
                #if sentence_str[0].isdigit():
                #    continue
                doc = nlp(sentence_str)
                # Get the bad tokens
                non_eng = [token.is_oov for token in doc].count(True)
                # Continue if more than 3
                if  non_eng > 3:
                    continue
                # Continue if the ratio is bad
                if non_eng != 0:
                    if non_eng / len(doc) > .2:
                        continue
                        
                # Remove UNICODE
                #sentence_str = sentence_str.encode("ascii", "ignore")
                #sentence_str = sentence_str.decode()
                # If description append
                if SpanPredictor(sentence_str):
                    data[species].append(sentence_str)
                    print(sentence_str, URL)
        except: 
            continue

In [ ]:
data